In [99]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from sklearn.metrics import r2_score
#import seaborn
#seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
import matplotlib.ticker
%matplotlib qt
# %matplotlib inline
import datetime as dt
pd.set_option('display.max_rows', 1000) 

## S4 H2 adsorption isotherm, lumped injections

### Reading in the data and preparing DataFrames

In [100]:
#reading vaclogger measurement file
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()

#adding an elapsed time column
vac_timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(vac_timestamps-vac_timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
# vaclog.drop(index=vaclog.index[0], 
#         axis=0, 
#         inplace=True)
vaclog["Time"] = vac_timestamps.dt.strftime('%d-%m-%Y %H:%M:%S')
vaclog["Time"] = pd.to_datetime(vaclog["Time"],format='%d-%m-%Y %H:%M:%S')

print(vaclog)


                                     Live comments                Time  \
0                                              NaN 2023-02-23 12:38:12   
1     inj volume prepared, baseline mass scan done 2023-02-23 12:38:19   
2                                              NaN 2023-02-23 12:38:26   
3                                              NaN 2023-02-23 12:38:33   
4                                              NaN 2023-02-23 12:38:40   
...                                            ...                 ...   
2042                                           NaN 2023-02-23 16:31:35   
2043                                           NaN 2023-02-23 16:31:41   
2044                                           NaN 2023-02-23 16:31:49   
2045                                           NaN 2023-02-23 16:31:55   
2046                                           NaN 2023-02-23 16:32:02   

      Elapsed time  injection 100mbar      Barion_2      Barion_1  \
0              0.0             88.984  7.7

### Data processing

In [101]:
#print vaclog comments
print(pd.unique(vaclog["Live comments"]))

[nan 'inj volume prepared, baseline mass scan done' 'RGA filament off'
 'turbo off' 'bp check']


#### CernOx temperature R-T conversion

In [102]:
#Temperature curve for CERNOX 
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
# specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


## Ignore end of measurement

In [103]:
#slicing the dataset
ramp = vaclog.iloc[list((vaclog["Elapsed time"]/3600).between(0,2))]
print(ramp)

                                     Live comments                Time  \
0                                              NaN 2023-02-23 12:38:12   
1     inj volume prepared, baseline mass scan done 2023-02-23 12:38:19   
2                                              NaN 2023-02-23 12:38:26   
3                                              NaN 2023-02-23 12:38:33   
4                                              NaN 2023-02-23 12:38:40   
...                                            ...                 ...   
1050                                           NaN 2023-02-23 14:37:44   
1051                                           NaN 2023-02-23 14:37:51   
1052                                           NaN 2023-02-23 14:37:58   
1053                                           NaN 2023-02-23 14:38:05   
1054                                           NaN 2023-02-23 14:38:11   

      Elapsed time  injection 100mbar      Barion_2      Barion_1  \
0              0.0             88.984  7.7

## Subtracting baseline

In [104]:
#TODO - calculating coverages for plotting pressure v coverage
#in the first assumption coverage is assumed to be equivalent to the number of molecules injected
#coverage at any given time point in [M/cm2] -> M/s * total injection time 
#First - Determine the inj start point, set elapsed time here to 0. 
inj = vaclog.loc[vaclog["Barion_2"]/0.46 >= 9e-8]
inj.drop(columns="Elapsed time", inplace=True)
print(inj)
timestamps2 = pd.to_datetime(inj["Time"],format="%d/%m/%Y %H:%M:%S")
runtime2 = (timestamps2-timestamps2[92]).dt.total_seconds()    #hardcoded value
inj.insert(1,"Elapsed time",runtime2)



     Live comments                Time  injection 100mbar      Barion_2  \
92             NaN 2023-02-23 12:48:48             88.986  1.800000e-07   
93             NaN 2023-02-23 12:48:55             88.985  1.300000e-07   
94             NaN 2023-02-23 12:49:02             88.985  1.890000e-07   
95             NaN 2023-02-23 12:49:09             88.985  2.670000e-07   
96             NaN 2023-02-23 12:49:16             88.981  3.110000e-07   
...            ...                 ...                ...           ...   
2042           NaN 2023-02-23 16:31:35             83.563  2.660000e-06   
2043           NaN 2023-02-23 16:31:41             83.562  2.660000e-06   
2044           NaN 2023-02-23 16:31:49             83.563  2.660000e-06   
2045           NaN 2023-02-23 16:31:55             83.563  2.660000e-06   
2046           NaN 2023-02-23 16:32:02             83.566  2.660000e-06   

          Barion_1  DUAL experiment  DUAL insulation  injection 1mbar  helium  \
92    1.830000e-07

C:\Users\etiirine\AppData\Local\Temp\ipykernel_8908\1257053034.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inj.drop(columns="Elapsed time", inplace=True)


In [105]:
#TODO - subtract the baseline from graph 2 (linear pressure scale) and plot again
#adding H2 equiv pressure readings
inj.insert(3, "corr BA2", inj["Barion_2"]/1)#0.46)
inj.insert(4, "corr BA1", inj["Barion_1"]/1)#0.46)


### Thermal transpiration
Applied for the gauge readings using the formula below:
    $$
    \frac{p_2}{p_1}=\sqrt \frac{T_2}{T_1}
    $$
Here $p_2$ and $T_2$ are the pressure and temperature in the cold part and $p_1$, $T_1$ are the pressure, temperature the gauge is exposed to.

In [106]:
#implementing the formula
T2 = 4.2
T1 = 293
p_coef = 1#np.sqrt(T2/T1)
inj.insert(3, "H2 4K BA2", (inj["corr BA2"]*p_coef))
inj.insert(4, "H2 4K BA1", (inj["corr BA1"]*p_coef))


In [107]:
print(p_coef)
inj.head()

1


,Live comments,Elapsed time,Time,H2 4K BA2,H2 4K BA1,corr BA2,corr BA1,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
92,NaN,0.0,2023-02-23 12:48:48,1.800000e-07,1.830000e-07,1.800000e-07,1.830000e-07,88.986,1.800000e-07,1.830000e-07,1.026000e-07,3.718000e-07,1.1,451.0,-5.387,4273.384,NaN,NaN
93,NaN,7.0,2023-02-23 12:48:55,1.300000e-07,1.360000e-07,1.300000e-07,1.360000e-07,88.985,1.300000e-07,1.360000e-07,1.378000e-07,3.714000e-07,1.1,451.0,-5.275,4273.384,NaN,NaN
94,NaN,14.0,2023-02-23 12:49:02,1.890000e-07,2.000000e-07,1.890000e-07,2.000000e-07,88.985,1.890000e-07,2.000000e-07,1.971000e-07,3.718000e-07,1.1,451.0,-5.275,4273.496,NaN,NaN
95,NaN,21.0,2023-02-23 12:49:09,2.670000e-07,2.760000e-07,2.670000e-07,2.760000e-07,88.985,2.670000e-07,2.760000e-07,2.297000e-07,3.718000e-07,1.1,451.0,-5.275,4273.496,NaN,NaN
96,NaN,28.0,2023-02-23 12:49:16,3.110000e-07,3.430000e-07,3.110000e-07,3.430000e-07,88.981,3.110000e-07,3.430000e-07,3.930000e-07,3.718000e-07,1.1,450.0,-5.050,4273.608,NaN,NaN


In [108]:
#define baseline assuming that pressure during injection >> base pressure
base1 = inj[inj["Barion_1"].between(1e-8,2e-7)] #NB! hardcoded
base2 = inj[inj["Barion_1"].between(1e-8,2e-7)]
print(base2)
#take the average of the baseline
avg1 = np.mean(base2["corr BA1"])
avg2 = np.mean(base1["corr BA2"])

print(avg1)
print(avg2)


   Live comments  Elapsed time                Time     H2 4K BA2  \
92           NaN           0.0 2023-02-23 12:48:48  1.800000e-07   
93           NaN           7.0 2023-02-23 12:48:55  1.300000e-07   
94           NaN          14.0 2023-02-23 12:49:02  1.890000e-07   

       H2 4K BA1      corr BA2      corr BA1  injection 100mbar      Barion_2  \
92  1.830000e-07  1.800000e-07  1.830000e-07             88.986  1.800000e-07   
93  1.360000e-07  1.300000e-07  1.360000e-07             88.985  1.300000e-07   
94  2.000000e-07  1.890000e-07  2.000000e-07             88.985  1.890000e-07   

        Barion_1  DUAL experiment  DUAL insulation  injection 1mbar  helium  \
92  1.830000e-07     1.026000e-07     3.718000e-07              1.1   451.0   
93  1.360000e-07     1.378000e-07     3.714000e-07              1.1   451.0   
94  2.000000e-07     1.971000e-07     3.718000e-07              1.1   451.0   

    T-platinum  T-CERNOX  I_emission  I_grid  
92      -5.387  4273.384         NaN  

In [109]:
#subtract the baseline from the graph to get the actual pressure evolution on the sample

#Plotting the h2 equiv base pressure subtracted data
fig, ax = plt.subplots()
#subtract the baseline from the graph to get the actual pressure evolution on the sample

ax.plot(inj["Time"],inj['H2 4K BA1'].sub(avg1),marker=".", markersize=5,label='BA1')
ax.plot(inj["Time"],inj['H2 4K BA2'].sub(avg2),marker=".", markersize=5,label='BA2')
plt.legend(loc="upper left")

ax.set_xlabel('Timestamp')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('linear')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    
       
plt.legend(loc="upper right")
plt.title("Pressure evolution on the sample, p increase from inj flux subtracted. \n Pressures H2 equivalent, thermal transp. corrected ")


Text(0.5, 1.0, 'Pressure evolution on the sample, p increase from inj flux subtracted. \n Pressures H2 equivalent, thermal transp. corrected ')

### Injection pressure linearity

In [110]:
#defining a function for the straight line
def func(x,a,b):
    return a*x + b
#curve fit for the data
y = inj["injection 100mbar"].values
x = (inj["Elapsed time"]/3600).values
params, cov = curve_fit(func,x,y)
#straight line parameters
a, b = params
print(f"Fitted line: y = {a:.2f}*x+{b:.2f}")

Fitted line: y = -0.58*x+85.09


### Formula for sum of the differential injection flux
$$
N=\frac{\Sigma dp\cdot V_{inj}}{k_B\cdot T}=\frac{\Sigma (p_{inj,t(x)}-p_{inj,t(x-1)})\cdot V_{inj}}{k_B \cdot T}   \space \left[{M}\right]
$$

In [111]:
#constants in SI units
T = 293
k_B = 1.38e-23
V_inj = 6.515e-5
S_sample= 276
#implement the formula above
dp = abs(np.diff(inj["injection 100mbar"]*100))
print(np.mean(dp))
dp = np.insert(dp,0,0)
N_x = (dp.cumsum())*V_inj/(k_B*T)
print(N_x)
N_cov = (N_x/S_sample)
print(N_cov)


0.3423746161720071
[0.00000000e+00 1.61126774e+15 1.61126774e+15 ... 1.07745474e+19
 1.07745474e+19 1.07793812e+19]
[0.00000000e+00 5.83792661e+12 5.83792661e+12 ... 3.90382153e+16
 3.90382153e+16 3.90557290e+16]


In [112]:
inj.insert(2, "Coverage",N_cov)


In [113]:
inj.head()

,Live comments,Elapsed time,Coverage,Time,H2 4K BA2,H2 4K BA1,corr BA2,corr BA1,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
92,NaN,0.0,0.000000e+00,2023-02-23 12:48:48,1.800000e-07,1.830000e-07,1.800000e-07,1.830000e-07,88.986,1.800000e-07,1.830000e-07,1.026000e-07,3.718000e-07,1.1,451.0,-5.387,4273.384,NaN,NaN
93,NaN,7.0,5.837927e+12,2023-02-23 12:48:55,1.300000e-07,1.360000e-07,1.300000e-07,1.360000e-07,88.985,1.300000e-07,1.360000e-07,1.378000e-07,3.714000e-07,1.1,451.0,-5.275,4273.384,NaN,NaN
94,NaN,14.0,5.837927e+12,2023-02-23 12:49:02,1.890000e-07,2.000000e-07,1.890000e-07,2.000000e-07,88.985,1.890000e-07,2.000000e-07,1.971000e-07,3.718000e-07,1.1,451.0,-5.275,4273.496,NaN,NaN
95,NaN,21.0,5.837927e+12,2023-02-23 12:49:09,2.670000e-07,2.760000e-07,2.670000e-07,2.760000e-07,88.985,2.670000e-07,2.760000e-07,2.297000e-07,3.718000e-07,1.1,451.0,-5.275,4273.496,NaN,NaN
96,NaN,28.0,2.918963e+13,2023-02-23 12:49:16,3.110000e-07,3.430000e-07,3.110000e-07,3.430000e-07,88.981,3.110000e-07,3.430000e-07,3.930000e-07,3.718000e-07,1.1,450.0,-5.050,4273.608,NaN,NaN


In [114]:
## dropping coverage values close to 0
inj_new = inj.loc[inj['Coverage'] > 1e+13]
inj_new.head()
#inj_new["H2 4K BA2"].sub(avg2).describe()

,Live comments,Elapsed time,Coverage,Time,H2 4K BA2,H2 4K BA1,corr BA2,corr BA1,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
96,NaN,28.0,2.918963e+13,2023-02-23 12:49:16,3.110000e-07,3.430000e-07,3.110000e-07,3.430000e-07,88.981,3.110000e-07,3.430000e-07,3.930000e-07,3.718000e-07,1.1,450.0,-5.050,4273.608,NaN,NaN
97,NaN,34.0,2.918963e+13,2023-02-23 12:49:22,5.170000e-07,5.230000e-07,5.170000e-07,5.230000e-07,88.981,5.170000e-07,5.230000e-07,4.458000e-07,3.714000e-07,1.1,450.0,-5.387,4273.608,NaN,NaN
98,NaN,41.0,7.589305e+13,2023-02-23 12:49:29,5.970000e-07,6.070000e-07,5.970000e-07,6.070000e-07,88.973,5.970000e-07,6.070000e-07,5.032000e-07,3.718000e-07,1.1,450.0,-5.275,4273.608,NaN,NaN
99,NaN,48.0,1.225965e+14,2023-02-23 12:49:36,8.880000e-07,1.050000e-06,8.880000e-07,1.050000e-06,88.965,8.880000e-07,1.050000e-06,1.432000e-06,3.718000e-07,1.1,451.0,-5.163,4273.608,NaN,NaN
100,NaN,55.0,2.160033e+14,2023-02-23 12:49:43,2.050000e-06,2.080000e-06,2.050000e-06,2.080000e-06,88.949,2.050000e-06,2.080000e-06,1.600000e-06,3.718000e-07,1.1,450.0,-5.275,4273.720,NaN,NaN


# Plotting

In [115]:
#plotting parameters
font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 10,
        }

plt.rcParams["figure.figsize"] = [8, 5]
plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.grid"] = True
plt.rcParams["grid.color"] = "black"
plt.rcParams["grid.linewidth"] = 0.35
plt.rcParams["axes.grid.axis"] = "both"
plt.rcParams["axes.grid.which"] = "both"



In [116]:
#Plotting the general pressure evolution and temperature for overview
fig, ax = plt.subplots()

l5 = ax.plot(vaclog["Time"], polyfit(A,vaclog["T-CERNOX"]),marker=".", color="cyan", markersize=5,label='Temperature')
ax.set_ylabel('Temperature (K)')
ax.set_xlabel('Timestamp')

# Format the x-axis as timestamps
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))

# Rotate and align the x-axis tick labels
fig.autofmt_xdate(rotation=40, ha='right')

#Duplicate x axis for pressure values
ax2=ax.twinx()
l1 = ax2.plot(vaclog["Time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
l2 = ax2.plot(vaclog["Time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
l3 = ax2.plot(vaclog["Time"],vaclog['DUAL experiment'],marker=".", markersize=4,label='Dual gauge')
l4 = ax2.plot(vaclog["Time"],vaclog['injection 100mbar'],marker=".", markersize=4,label='100mbar F-R')
ax2.set_ylabel('Pressure (mbar)')
ax2.legend(handles = l1+l2+l3+l4+l5, loc="upper right")
ax2.set_yscale('log')

plt.title("Pressure evolution + temperatures")
plt.tight_layout()  # Ensure all elements are properly placed within the figure
plt.savefig(r'./graphs/S4 a-C H2 adsorption isotherm_overview.png', dpi=300)
plt.show()


In [117]:
#Plotting BA2, BA1 pressure
fig, ax = plt.subplots()

ax.plot(vaclog["Time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
ax.plot(vaclog["Time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
ax.plot(vaclog["Time"],vaclog["DUAL experiment"],marker=".", markersize=4,label='DUAL experiment')
ax.plot(vaclog["Time"],vaclog['injection 100mbar'],marker=".", markersize=4,label='100mbar F-R')

ax.set_xlabel('Timestamp')
ax.set_ylabel('Pressure (mbar)')
ax.legend()
ax.set_title("S4 CU ESD sample out - Total pressure evolution")

#ax.grid(which='major', axis='y')
#ax.get_yaxis().get_major_formatter().labelOnlyBase = False
#ax.tick_params(axis="y", which='major')
ax.set_yscale('log')

# Format the x-axis as timestamps
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))

# Rotate and align the x-axis tick labels
fig.autofmt_xdate(rotation=40, ha='right')

plt.tight_layout()  # Ensure all elements are properly placed within the figure
plt.savefig(r'./graphs/S4 a-C H2 adsorption isotherm_Pressures.png',dpi=300)
plt.show()

In [118]:
#plotting the sliced data
#Plotting the general pressure evolution for overview
fig, ax = plt.subplots()

ax.plot(ramp["Time"],ramp['Barion_1'],marker=".", markersize=4,label='BA1')
ax.plot(ramp["Time"],ramp['Barion_2'],marker=".", markersize=4,label='BA2')
ax.plot(ramp["Time"],ramp['DUAL experiment'],marker=".", markersize=4,label='DUAL gauge')

ax.legend(loc="upper right")

ax.set_xlabel('Timestamp')
ax.set_ylabel('Pressure (mbar)')
ax.legend(loc="upper right")
ax.set_title("Pressure evolution during injection. Corrected for H2")

# Format the x-axis as timestamps
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))

# Rotate and align the x-axis tick labels
fig.autofmt_xdate(rotation=40, ha='right')

plt.tight_layout()  # Ensure all elements are properly placed within the figure       
plt.savefig(r'./graphs/S4 a-C H2 adsorption isotherm_Sliced df Pressures.png',dpi=300)
plt.show()


In [119]:
#checking inj pressure linearity, graph
fig, ax = plt.subplots()
y_fit = func(x,a,b)
ax.plot(inj["Time"],inj["injection 100mbar"],marker=".", markersize=5,label='inj volume pressure')
ax.legend(loc="upper right")
ax.set_xlabel('Timestamp')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('linear')

# Format the x-axis as timestamps
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))

# Rotate and align the x-axis tick labels
fig.autofmt_xdate(rotation=40, ha='right')

plt.title("Injection pressure evolution 100mbar conductance gauge")
plt.tight_layout()  # Ensure all elements are properly placed within the figure       
plt.savefig(r'./graphs/S4 a-C H2 adsorption isotherm_Sliced df Pressures.png',dpi=300)
plt.show()

In [120]:
#plot coverage vs time
fig, ax = plt.subplots()

ax.plot(inj["Time"],inj["Coverage"],marker=".", markersize=5,label='Coverage')
ax.legend(loc="upper right")
ax.set_xlabel('Timestamp')
ax.set_ylabel('Coverage (M/s*cm2)')
ax.set_yscale('linear')

# Format the x-axis as timestamps
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))

# Rotate and align the x-axis tick labels
fig.autofmt_xdate(rotation=40, ha='right')

plt.title("Coverage over time")
plt.tight_layout()  # Ensure all elements are properly placed within the figure       
plt.savefig(r'./graphs/S4 a-C H2 ads isotherm_coverage vs time.png',dpi=300)
plt.show()

In [121]:
#plotting pressure vs coverage
fig, ax = plt.subplots()

ax.plot((inj_new["Coverage"]),inj_new["H2 4K BA2"].sub(avg2),marker=".", markersize=5,label='isotherm')
ax.legend(loc="upper left")
#xrange = np.arange(min(inj_new["Coverage"]),max(inj_new["Coverage"]),5e+15)
#ax.set_xticks(xrange)
ax.set_xlim(left = 5e14,right = 3.119e16)
ax.set_ylim(bottom=0)

ax.set_xlabel('Coverage (M/cm2)')
ax.set_ylabel('Pressure (mbar)')

plt.title("H2 adsorption isotherm with lumped injection")
plt.tight_layout()  # Ensure all elements are properly placed within the figure       
plt.savefig(r'./graphs/S4 a-C H2 adsorption isotherm.png',dpi=300)
plt.show()

## Writing edited dataframes to file


In [122]:
#desorption yield params
#vaclog
vaclog.to_csv(r'./calculated datasets/vaclogger_output.csv', sep='\t', encoding='utf-8', header=True)

#CERNoX
temps_dict = {"Timestamp":vaclog["Time"],"Temperatures":polyfit(A,vaclog["T-CERNOX"])}
temps = pd.DataFrame(temps_dict)
temps.to_csv(r'./calculated datasets/CERNoX temps.csv', sep='\t', encoding='utf-8', header=True)
#coverage
inj.to_csv(r'./calculated datasets/vaclog during injection.csv', sep='\t', encoding='utf-8', header=True)

#coverage > 1e13
inj_new.to_csv(r'./calculated datasets/cut vaclog during injection.csv', sep='\t', encoding='utf-8', header=True)
